# Módulo 1: Análisis de datos en el ecosistema Python

### Sesión (24)

# Análisis de series temporales (Time Series)

El análisis de series temporales (__Time Series Analysis__) es el estudio de puntos de datos recopilados a lo largo del tiempo, normalmente en intervalos fijos. El objetivo es comprender los __patrones__ y __tendencias__ subyacentes en los datos y utilizar esta información para hacer __predicciones__ sobre valores futuros.

![time-series.png](attachment:time-series.png)

Para realizar un análisis de series temporales en _Python_, generalmente podemos usar ***pandas*** para cargar, manipular y visualizar los datos, y la librería ***statsmodels*** para aplicar diferentes técnicas comunes como el **análisis de tendencias**, el **análisis de estacionalidad** y de **autocorrelacións** que nos sirven de ayuda de cara a la creación y evaluación de **modelos predictivos**.

![statsmodels.png](attachment:statsmodels.png)

In [ ]:
# importamos las librerías necesarias
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
# Modificamos los parámetros de los gráficos en matplotlib
from matplotlib.pyplot import rcParams

rcParams['figure.figsize'] = 12, 6 # el primer dígito es el ancho y el segundo el alto
rcParams["font.weight"] = "bold"
rcParams["font.size"] = 10
rcParams["axes.labelweight"] = "bold"

### Airline Passenger Dataset

Importamos los datos del ejemplo disponible en la librería _statsmodels_

In [ ]:
import statsmodels.api as sm

# Cargar el dataset
df_air = sm.datasets.get_rdataset('AirPassengers').data

print("Tipo de dataset:", type(df_air))
df_air

Tipo de dataset: <class 'pandas.core.frame.DataFrame'>


,time,value
0,1949.000000,112
1,1949.083333,118
2,1949.166667,132
3,1949.250000,129
4,1949.333333,121
...,...,...
139,1960.583333,606
140,1960.666667,508
141,1960.750000,461
142,1960.833333,390


In [ ]:
plt.plot(df_air)
plt.show()

In [ ]:
sns.lineplot(data=df_air, x='time', y='value')
plt.show()

Como podemos observar, la columna _`time`_ contiene los marcadores mensuales:

In [ ]:
print(1/12, 2/12, 3/12, '...')

Una técnica común consiste en convertir o definir el tiempo registrado o los _timestamps_ en un objeto _pandas **DateTimeIndex**_, lo que nos permite realizar operaciones basadas en el tiempo, y tener los índices para cambiar o manipular fácilmente los datos de la serie temporal.

Viendo el contenido de la columna _`time`_, creamos un objeto _DatetimeIndex_ usando la función `pd.date_range()`, que genera un rango de fechas con una frecuencia mensual (`freq='MS'`) entre la fecha de inicio de `'1949-01-01'` y la fecha de finalización de `'1960-12-01'`.

In [ ]:
# Definir un rango de fechas con pasos fijos
fechas = pd.date_range(start='1949-01-01', end='1960-12-01', freq='MS')

# Asignar a las fechas como los índices del DataFrame
df_air.set_index(fechas, inplace=True)

# Quitar la columna 'time' de la tabla
df_air.drop(columns='time', inplace=True)

df_air

In [ ]:
# Visualizamos los datos preparados
plt.plot(df_air)
plt.show()

La gráfica de una serie temporal como otros conjuntos de datos está formado por puntos, a pesar de que se visualizan habitualmente de forma de líneas.

In [ ]:
# Gráfico de puntos de datos
sns.scatterplot(data=df_air)
plt.show()

In [ ]:
# Consultar los índices creados
df_air.index

Otra forma de cargar los datos de este ejemplo es mediante la librería ___seaborn___

In [ ]:
import seaborn as sns
import pandas as pd

# Cargar el dataset de "flights"
df_flight = sns.load_dataset('flights')

print("Tipo de dataset:", type(df_flight))
df_flight

In [ ]:
# Consultar la información del dataset descargado
df_flight.info()

Vemos que este dataset contiene mismos valores, pero el tiempo viene registrado en forma de **_año_ y _mes_ por separado**. En este caso podemos juntar esta información en una nueva columna y convertirla en ___DatetimeIndex___

In [ ]:
# Crear una nueva columna con la unificación de otras dos
df_flight['year_month'] = df_flight.apply(lambda x: str(x['year']) + '-' + x['month'], axis=1)

df_flight

In [ ]:
# Convertir la columna en fechas tipo DatetimeIndex
df_flight['fechas'] = pd.to_datetime(df_flight['year_month'], format='%Y-%b')

# Convertir la columna de fechas a los índices del DataFrame
df_flight.set_index('fechas', inplace=True)

# Quitar las columnas no necesarias
df_flight.drop(columns=['year', 'month', 'year_month'], inplace=True)

df_flight

In [ ]:
# Visualizar el DataFrame
plt.plot(df_flight)
plt.show()

Vamos a comprobar si las dos formas de cargar y tratar los datos nos han llevado al mismo Dataset:

In [ ]:
# Consultar las dos tablas
display(df_air)
display(df_flight)

Para comparar dos _DataFrames_ podemos usar el método `.equals()` que nos permite comprobar si dos objetos de este tipo son **exactamente identicos** o no:

In [ ]:
# Comparar las dos tablas (DataFrames)
df_air.equals(df_flight)

Sabemos que la **diferencia** está en los **nombres de las columnas** y el **índice** y **no en el contenido**. Podemos utilizar la función `array_equal()` de _numpy_ para asegurarnos que los dos tablas contienen la misma información:

In [ ]:
# Comparar los valores
np.array_equal(df_air.values, df_flight.values)

In [ ]:
# Comparar los índices
np.array_equal(df_air.index, df_flight.index)

In [ ]:
# Comparación visual
fig, axes = plt.subplots(2,1, figsize=(16,9))
sns.lineplot(data=df_air, ax=axes[0])
sns.lineplot(data=df_flight, ax=axes[1])
plt.show()

De las principales ventajas de tener los tiempos registrados como un objeto _DatetimeIndex_ podemos mencionar:
- **Resampling**
- **Time-based slicing**
- **Time zone handling**

In [ ]:
# Podemos hacer un remuestreo por ejemplo para agrupar los datos por año, promediando los valores mensuales
df_air_anual = df_air.resample('Y').mean()
display(df_air_anual)
plt.plot(df_air_anual)
plt.show()

In [ ]:
# Podemos seleccionar fácilmente los registros en un intervalo concreto entre dos fechas por ejemplo
df_air['1954-03-30':'1955-02-10']

In [ ]:
# Podemos llevar la hora internacional (UTC) a nuestra zona horaria local "Central European Time (CET)"
df_air.tz_localize('Europe/Madrid', ambiguous='NaT')

### Visualización más avanzada y dinámica con plotly

**[plotly](https://plotly.com/python/)** es una librería popular de visualización de datos de código abierto en _Python_, _R_ y otros lenguajes de programación que permite crear gráficos y Dashboards totalmente **interactivos**, lo que permite a los usuarios acercar y alejar, desplazar y pasar el ratón sobre los puntos de datos para ver más información.  

_Plotly_ Proporciona una **amplia gama de tipos de gráficos** y opciones de **personalización**, lo que la convierte en una herramienta versátil para la visualización de datos.

In [ ]:
# Las gráficas interactivas ayudan bastante a los analistas de datos,
# sobre todo a la hora de analizar y explorar las series temporales
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_air.index,
    y=df_air['value'],
    mode="markers+lines"
    ))

fig.update_layout(title="Número de pasajeros aéreos de cada mes desde el año 1949 al 1960",
                  title_font_size=26,
                  xaxis_title = 'Fecha',
                  yaxis_title= 'Pasajeros'
                  )

fig.show()

### Analizar la estacionalidad

El análisis de estacionalidad (___Seasonality analysis___) es el proceso de **identificar patrones** en los datos que **se repiten durante intervalos fijos de tiempo**, por ejemplo en un **año** o un **trimestre**. Implica examinar los datos de la serie temporal para determinar si existen **ciclos regulares** o patrones que ocurren dentro de los datos durante un **período específico**. Esto puede ayudar a predecir tendencias futuras y a tomar decisiones basadas en los datos históricos.

En este ejemplo podemos mirar como los datos muestran una **periodicidad anual** por poseer una **estacionalidad considerable** sobre los niveles de datos mensuales.

In [ ]:
# Podemos directamente pintar los datos por tramos anuales
sns.lineplot(data=df_air, x=df_air.index, y=df_air['value'], hue=df_air.index.year, palette='husl')
plt.show()

#### Graficar el _Seasonal plot_

Una técnica común para estudiar y explorar los patrones estacionales en los datos es sacar el ___Seasonal plot___ que consiste en **superponer** las gráficas que visualizan la **evolución de los datos en el periodo marcado** por la estacionalidad.

In [ ]:
# Crear una tabla pivotada en base a la columna que contiene los datos estacionales
df_piv = pd.pivot_table(df_air,                      # Dataframe principal
                        index=df_air.index.month,    # Las unidades del eje horizontal
                        columns=df_air.index.year,   # El periodo estacional
                        values=['value'])            # La columna que contiene los datos dentro del DataFrame
df_piv

In [ ]:
plt.figure(figsize=(17,10))
plt.plot(df_piv, label=df_piv.columns)
plt.title('Número de pasajeros aéreos (mensuales)', fontsize=16)
plt.xlabel('Meses')
plt.ylabel('Pasajeros')
plt.legend(loc='upper right')
plt.show()

La otra alternativa sería graficar con la librería _seaborn_ todas las columnas mediante un bucle para llegar a tener el gráfico de _seasonal plot_.

In [ ]:
plt.figure(figsize=(17,10))

# Un bucle para dibujar las columnas que corresponden a cada año
for col in range(df_piv.shape[1]):
    sns.lineplot(data=df_piv, x=df_piv.index, y=df_piv.iloc[:,col].values, label=df_piv.columns[col])

plt.title('Número de pasajeros aéreos (mensuales)', fontsize=16)
plt.xlabel('Meses')
plt.ylabel('Pasajeros')
plt.legend(loc='upper right')

plt.show()

Como muestra el gráfico de _seasonal plot_, este dataset cuenta con un **componente estacional** que innegablemente afecta a los números de pasajeros aéreos en función del mes de año, muy probablamente por los **periodos vacacionales**, el **clima** y los **ciclos económicos** a lo largo de un año.

### Descomposición de la serie temporal

Otra técnica usada ampliamente en el análisis de la estacionalidad de una serie temporal es la descomposición de la misma o el ___Time series decomposition___ que pretende descomponer una serie temporal en sus **componentes subyacentes (_tendencia_, _estacional_ y _residual_)** con el fin de comprender y analizar mejor los datos. Los tres componentes de la descomposición de series de tiempo son:

- **Trend** (tendencia) : esta es la **dirección a largo plazo** en la que se mueve la serie temporal. Captura el comportamiento general de la serie durante un largo período de tiempo. Puede ser **creciente**, **decreciente** o **plano**.

- **Seasonality** (estacionalidad): Este es el **patrón de fluctuaciones recurrentes** o ciclos en la serie de tiempo que se repiten a intervalos regulares.

- **Residual**: Esta es la **variación aleatoria** en la serie de tiempo que no puede explicarse por la tendencia o la estacionalidad. Representa el **ruido o error** en los datos.

La descomposición de series temporales se puede realizar utilizando dos enfoques diferentes:  
- Descomposición **aditiva**:  La serie temporal se descompone en la **suma de sus componentes** de tendencia, estacionalidad y residuos.
- Descomposición **multiplicativa**:  La serie temporal se descompone en el **producto de sus componentes** de tendencia, estacionalidad y residuos.

Una vez que se descompone la serie temporal, cada componente se puede analizar por separado para obtener información sobre los datos. Por ejemplo, el componente de **tendencia** se puede usar para identificar **patrones a largo plazo** o cambios incrementales en los datos, mientras que el componente **estacional** se puede usar para identificar **ciclos regulares** o patrones en los datos.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

df_air_des = seasonal_decompose(df_air, model='additive')

display(df_air_des.trend[100:105])
display(df_air_des.seasonal[100:105])
display(df_air_des.resid[100:105])

In [ ]:
# Establecer el tamaño de la imégen
rcParams['figure.figsize'] = 18, 8

# Graficar la descomposición de la serie temporal
df_air_des.plot()
plt.show()

# Podemos volver a resetear los tamaños a los valores por defecto
# plt.rcdefaults()

Ahora sumamos la tendencia y el componente estacional para observar los patrones extraídos con esta descomposición:

In [ ]:
rcParams['figure.figsize'] = 12, 6
plt.plot(df_air_des.seasonal + df_air_des.trend)
plt.title("seasonal_decompose:  Seasonal + Trend")
plt.show()

Al visualizar el periodo anual mediante _`seasonal plot`_ sabemos que el **patrón estacional tiene una forma cada vez más estirada**. Se puede observar que **al fijar la forma de componente estacional**, la evolución de la serie temporal **no se refleja correctamente** en la suma de estos dos componentes. Visualizamos mejor el componente residual que contiene la parte no explicada de la serie:

In [ ]:
display(df_air_des.resid.describe().round(2))
plt.plot(df_air_des.resid)
plt.title("seasonal_decompose:  Residual")
plt.show()

Vemos que claramente hay un **comportamiento periódico** en estos datos que indica que todavía **queda una parte de información** que no se ha llegado a explotar.

#### Seasonal and Trend decomposition using Loess (_STL_)

La descomposición de tendencia estacional mediante Loess (**STL**) es un método avanzado de descomposición de series temporales que utiliza **modelos de regresión** ajustados localmente para descomponer una serie temporal en componentes de tendencia, estacionales y restantes, permitiendo la **flexibilidad** para el componente **estacional**.

In [ ]:
# Descomponer la serie y graficarla
from statsmodels.tsa.seasonal import STL
df_air_stl = STL(df_air).fit()

display(df_air_stl.trend[100:105])
display(df_air_stl.seasonal[100:105])
display(df_air_stl.resid[100:105])


In [ ]:
# Establecer el tamaño de la imégen
rcParams['figure.figsize'] = 18, 8

# Graficar la descomposición de la serie temporal
df_air_stl.plot()
plt.show()

# Podemos volver a resetear los tamaños a los valores por defecto
# plt.rcdefaults()

Se puede apreciar que el componente **estacional** tiene **una ampliación a lo largo de tiempo** que es más ajustado a la realidad. Esto hace que la suma de los dos componentes represente **un comportamiento similar a la de serie** temporal.

In [ ]:
rcParams['figure.figsize'] = 12, 6
plt.plot(df_air_stl.seasonal + df_air_stl.trend)
plt.title("STL:  Seasonal + Trend")
plt.show()

Por consiguiente los valores residuales **no presentan un comportamiento o un patrón concreto**. Son valores **más pequeños** y mustran **más aleatoriedad** que la descomposición anterior.

In [ ]:
display(df_air_stl.resid.describe().round(2))
plt.plot(df_air_stl.resid)
plt.title("STL:  Residual")
plt.show()

### Análisis de autocorrelación

La autocorrelación (___Autocorrelation___) mide un conjunto de valores actuales contra un conjunto de valores pasados de una serie para ver **si se correlacionan** o no. Entonces, en lugar de medir la correlación entre dos variables aleatorias, estamos midiendo la **correlación entre una variable aleatoria contra sí misma**.

A continuación vamos a calcular la correlación entre dos vectores para tenerlo más claro:

In [ ]:
# Calcular mediante "pandas" el coeficiente de correlación linear entre dos Series (Pearson’s r)
A = pd.Series([1, 2, 3, 4, 5])
B = pd.Series([10, 20, 30, 40, 50])
C = pd.Series([-10, -20, -30, -40, -50])

X = pd.Series([99, -0.05, 100006.3, 888, 0.025])

print("La correlación entre A y B = ", A.corr(B))
print("La correlación entre A y C = ", A.corr(C))
print("La correlación entre A y X = ", A.corr(X))

Podemos ver que la correlación (en este ejemplo de tipo _Pearson_) es la **fuerza con la que dos variables están relacionadas entre sí**. Si el valor es **+1** las variables están perfectamente **correlacionadas positivamente**, y si es **-1** están perfectamente **correlacionadas negativamente** y en caso de ser igual a **0 no hay correlación**.

![Pearson-Correlation-Coefficient-Formula.jpg](attachment:Pearson-Correlation-Coefficient-Formula.jpg)

Podemos calcular la correlación de las observaciones de una serie temporal con observaciones anteriores, llamados retrasos (___lags___). Debido a que la correlación de las observaciones de la serie temporal se calcula con valores de la misma serie en momentos anteriores, esto se denomina **correlación serial** o ***autocorrelación***.

La gráfica de la autocorrelación de una serie temporal se denomina **función de autocorrelación**, o el acrónimo ***ACF***. Esta gráfica a veces se denomina **correlograma**:

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

rcParams['figure.figsize'] = 14, 7
plot_acf(df_air, lags=37)
plt.xticks(np.arange(37))
plt.ylim(-1.1,1.1)
plt.show()

Fijandonos en la evolución de los coeficientes de correlación para cada _lag_, vemos que:  

- El _lag_ 0 tiene una correlación perfecta de 1 porque estamos **correlacionando la serie temporal con una copia exacta de sí misma**.  

- Hay un **patrón cíclico** evidente cada **múltiplo de 12**, confirmando que tenemos una **estacionalidad anual** en nuestros datos.  

- La **fuerza de la correlación se disminuye** a medida que aumentan los retrasos o los _lags_. Esto indica que tenemos una **tendencia** en nuestros datos.

##### Intervalos de confianza (_Confidence intervals_)

De forma predeterminada se establecen unos **intervalos de confianza del 95 %** que se dibujan muchas veces como un **cono** y sugieren  que los valores de correlación fuera de este cono (la región azul) son muy probablemente una correlación y **no una casualidad estadística**. Por lo tanto, al construir un modelo predictivo, esta gráfica indica que **probablemente solo debería considerar hasta el _lag_ 13** de los valores anteriores debido a sus **importancias estadísticas**.


#### Partial Autocorrelation Function (PACF)

Una **autocorrelación parcial** es la relación entre una serie de temporal y sus observaciones en pasos de tiempo anteriores  (_lags_), **eliminando todos los efectos intermedios**.

Como _PACF_ mide **la relación directa de una observación con las observaciones anteriores**, muestra un decline más allá de los retrasos que no tengan importancia, y por lo tanto se utiliza a menudo **para encontrar el tope de los _lags_ a considerar** para modelizar las series temporales.

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf

plot_pacf(df_air, lags=37, method='ywm')
plt.xticks(np.arange(37))
plt.ylim(-1.1,1.1)
plt.show()

### Stationarity Analysis

La **estacionariedad** significa que las **características estadísticas** de un proceso que genera una serie temporal, **no cambian con el tiempo**.  

Una serie temporal **estacionaria** es aquella **cuyas propiedades no dependen del momento en que se observa la serie**. Por lo tanto, **las series con _tendencias_ o con _estacionalidad_ no son estacionarias**.  
Por otro lado, una serie de **ruido blanco (_white noise_) es estacionaria**: no importa cuándo la observes, muestra **las mismas propiedades estadísticas** en cualquier momento.

![Stationary-Time-Series.png](attachment:Stationary-Time-Series.png)

La estacionariedad (_stationarity_) es un concepto fundamental en el análisis de series temporales, porque indica que de alguna manera **la media, la varianza y la autocorrelación de una serie de tiempo estacionaria no cambia con el tiempo**.

Ahora comparamos desde este aspecto las dos **componentes residuales** calculadas anteriormente, cuando **idealmente tienen que ser series estacionarias**:

In [ ]:
# Visualizamos el componente residual calculada mediante "seasonal_decompose"
plt.plot(df_air_des.resid)
plt.title("seasonal_decompose:  Residual")
plt.show()

Estudiamos las propiedades estadísticas de esta serie en varios momentos para ver su evolución a lo largo del tiempo

In [ ]:
df_resid1 = pd.DataFrame(df_air_des.resid)
df_resid1_piv = pd.pivot_table(df_resid1,
                               index=df_resid1.index.month,
                               columns=df_resid1.index.year)
df_resid1_piv

In [ ]:
plt.plot(df_resid1_piv.mean().values, label='las medias por año')
plt.plot(df_resid1_piv.std().values, label='las variaciones estándares por año')
plt.axhline(y=0, color='r', linestyle='--')
plt.title("Características estadísticas: [seasonal_decompose: Residual]")
plt.legend()
plt.ylim(-31,31)
plt.show()

Por otro lado, analizamos la evolución de las cualidades estadísticas del componente residual calculado mediante el método _STL_

In [ ]:
# Visualizamos el componente residual calculada mediante "seasonal_decompose"
plt.plot(df_air_stl.resid)
plt.title("STL:  Residual")
plt.show()

In [ ]:
df_resid2 = pd.DataFrame(df_air_stl.resid)
df_resid2_piv = pd.pivot_table(df_resid2,
                               index=df_resid2.index.month,
                               columns=df_resid2.index.year)
df_resid2_piv

In [ ]:
plt.plot(df_resid2_piv.mean().values, label='las medias por año')
plt.plot(df_resid2_piv.std().values, label='las variaciones estándares por año')
plt.axhline(y=0, color='r', linestyle='--')
plt.title("Características estadísticas: [STL: Residual]")
plt.legend()
plt.ylim(-31,31)
plt.show()

Se puede apreciar que el componente residual obtenido por la aplicación del método _`STL`_ presenta una **estacionariedad mayor**. Podemos analizar la función de **autocorrelación** para estas dos series para confirmar que **cuál de ellas contienen menos información** que **no se haya eleminado todavía**:  

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

rcParams['figure.figsize'] = 14, 7
plot_acf(df_resid1.dropna(), lags=37)
plt.title("ACF [seasonal_decompose: Residual]")
plt.xticks(np.arange(37))
plt.ylim(-1.1,1.1)
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

rcParams['figure.figsize'] = 14, 7
plot_acf(df_resid2.dropna(), lags=37)
plt.title("ACF [STL: Residual]")
plt.xticks(np.arange(37))
plt.ylim(-1.1,1.1)
plt.show()

Efectivamente estas gráficas confirman **una mayor presencia de _tendencia_ y _estacionalidad_** para la serie temporal de valores residuales calculados mediante _`seasonal_decompose`_ y por lo tanto **una menor estacionariedad** comparando con la otra serie.  

###  Rolling Window Calculations

A pesar de que el método `pivot_table()` es una función potente en la librería _pandas_ y nos permite transformar y resumir los datos de un _DataFrame_,  existen otros métods como ___rolling window calculations___ para escanear y analizar las propiedades locales que tenemos en los datos.  

**Cálculos de ventana móvil** se utiliza principalmente en el procesamiento de señales (_signal processing_) y datos de series temporales. Esta técnica consiste en **considerar una ventana** de tamaño fijo (_k_) y luego **realizar alguna operación matemática** deseada en esa ventana.

![rolling-sum-pandas-2.png](attachment:rolling-sum-pandas-2.png)

`rolling()` es un método para los _pandas.DataFrames_ o _pandas.Series_ que crea un objeto de ventana móvil que se utiliza para aplicar una función a los datos dentro de cada ventana. La clase _`Rolling`_ tiene funciones matemáticas populares como `sum()`,` mean()` y otras funciones relacionadas implementadas. Por otra parte, a través del método `apply()`, se pueden realizar operaciones matemáticas personalizadas en una ventana móvil.

Echamos un vistazo aquí en uno de los componentes residuales que hemos analizado antes sus propiedades, mediante la agrupación de los datos de cada año:

In [ ]:
# La tabla poivotada de los datos residuales en columnas para cada año
df_resid1_piv

In [ ]:
# La media calculada por columna/año
df_resid1_piv.mean()

Calculamos la media de los valores para las ventanas móviles de tamaño `12` que contienen datos por año, indicando que puede hacer estos cálculos incluso con 1 valor no nulo (_non-NA value_):

In [ ]:
# El método devuelve simplemente un objeto de clase rolling
df_resid1.rolling(window=12, min_periods=1)

In [ ]:
# El método por defecto mueve con pasos de uno la venta móvil
df_resid1.rolling(window=12, min_periods=1).mean()

In [ ]:
# Usamos el "Slicing" para sacar los valores de ventanas sin solapamiento o "overlapping"
df_resid1.rolling(window=12, min_periods=1).mean().iloc[11::12]

Vemos que **conseguimos el mismo resultado** que obtenimos anteriormente con el método `pivot_table()`:

In [ ]:
# Crear un DataFrame con los dos conjunto de valores para compararlos más fácilmente
pd.DataFrame({
    'pivot_table': df_resid1_piv.mean().values,
    'rolling_window': df_resid1.rolling(window=12, min_periods=1).mean().iloc[11::12].values.reshape(-1)
})

In [ ]:
# Graficar las medias móviles para cada año
df_resid1.rolling(window=12, min_periods=1).mean().iloc[11::12].plot()
plt.ylim(-31,31)
plt.show()

In [ ]:
# Graficar la desviación estándr para cada año
df_resid1.rolling(window=12, min_periods=1).std().iloc[11::12].plot()
plt.ylim(-31,31)
plt.show()

### Ruido blanco (_White Noise_)

En el análisis de series temporales, el ruido blanco es **un tipo de proceso estocástico en el que los valores no están correlacionados y tienen una media y una varianza constantes a lo largo del tiempo**. El ruido blanco es un concepto importante porque proporciona una línea de base con la que se pueden comparar otras series temporales y también se suele utilizar como componente en modelos de series temporales más complejos.

Teóricamente, una serie de tiempo es ruido blanco si las **observaciones se distribuyen de forma independiente e idéntica con una media y una varianza constantes**. A raíz de esto, la función de autocorrelación es cero para todos los retrasos (_lags_) y la serie temporal es de tipo **`estacionaria`**.

![descarga.jpg](attachment:descarga.jpg)


En otras palabras, los valores del ruido blanco son **aleatorios** y **no hay ninguna relación entre la observación actual y cualquier observación pasada o futura**. Las características principales del ruido blanco incluyen:

- **Media constante (cero)**: La media de una serie temporal de ruido blanco es constante e idealmente igual a cero.
- **Varianza constante**: La varianza de una serie temporal de ruido blanco es constante en el tiempo.
- **Autocorrelación cero**: La función de autocorrelación de una serie temporal de ruido blanco es igual a cero para todos los retrasos distintos de cero.
- **Distribución concreta** (_Gaussiana_ o _uniform_): Normalmente se supone que los valores de una serie temporal de ruido blanco tienen una distribución normal, aunque esto no es estrictamente necesario y se permite señales con la distribución uniforme.

En la práctica, podemos usar el ruido blanco para:
- **Agregar** a otra serie **variaciones aleatorias** que representan las mediciones y los errores presentes en datos reales, sin que estén relacionadas con ningún patrón o tendencia subyacente en los datos.

- **Considerar la hipótesis nula** en pruebas estadísticas para determinar si una serie es **significativamente diferente del ruido aleatorio** o no.

Ahora procedemos a generar el **ruido blanco** mediante la técnica de `np.random.normal` que disponemos en _numpy_:

In [ ]:
# Fijamos la semilla
np.random.seed(111)

# Generamos valores aleatorios con la media "0" y la varianza "1"
ruido1 = np.random.normal(loc=0, scale=1, size=1000)
pd.Series(ruido1).plot()
plt.show()

In [ ]:
# Las principales propiedades de este ruido
pd.Series(ruido1).describe()

In [ ]:
# Comprobamos la ACF
from statsmodels.graphics.tsaplots import plot_acf

rcParams['figure.figsize'] = 14, 7
plot_acf(ruido1, lags=20)
plt.xticks(np.arange(20))
plt.ylim(-1.1,1.1)
plt.show()

In [ ]:
# Comprobamos la PACF
from statsmodels.graphics.tsaplots import plot_pacf

rcParams['figure.figsize'] = 14, 7
plot_pacf(ruido1, lags=20, method='yw')
plt.xticks(np.arange(20))
plt.ylim(-1.1,1.1)
plt.show()

Podemos usar el módulo de `random` de _python_ para crear el ruido blanco con la distribución _Gaussiana_:

In [ ]:
import random

# Fijamos la semilla
random.seed(222)

# Generar 1000 valores aleatorios con una distribución normal
ruido2 = [random.gauss(mu=0, sigma=1) for x in range(1000)]
pd.Series(ruido2).plot()
plt.show()

In [ ]:
# Las principales propiedades de este ruido
pd.Series(ruido2).describe()

In [ ]:
# Comprobamos la ACF
from statsmodels.graphics.tsaplots import plot_acf

rcParams['figure.figsize'] = 14, 7
plot_acf(ruido2, lags=20)
plt.xticks(np.arange(20))
plt.ylim(-1.1,1.1)
plt.show()

Podemos generar el ruido mediante **valores distribuidos uniformemente** usando el método `np.random.rand` de _numpy_

In [ ]:
ruido3 = np.random.rand(1000) - 0.5

pd.Series(ruido3).plot()
plt.show()

In [ ]:
# Las principales propiedades de este ruido
pd.Series(ruido3).describe()

In [ ]:
# Comprobamos la ACF
from statsmodels.graphics.tsaplots import plot_acf

rcParams['figure.figsize'] = 14, 7
plot_acf(ruido3, lags=20)
plt.xticks(np.arange(20))
plt.ylim(-1.1,1.1)
plt.show()

In [ ]:
# Comprobamos la PACF
from statsmodels.graphics.tsaplots import plot_pacf

rcParams['figure.figsize'] = 14, 7
plot_pacf(ruido3, lags=20, method='yw')
plt.xticks(np.arange(20))
plt.ylim(-1.1,1.1)
plt.show()

Para demostrar que el **ruido** es una serie **estacionaria** (_stationary_) podemos estudiar **la evolución de sus propiedades estadísticas** y comprobar si se modifican con el paso de tiempo.

In [ ]:
win = 100
ruido3_media = pd.Series(ruido3).rolling(win).mean().iloc[win-1::win]
ruido3_std = pd.Series(ruido3).rolling(win).std().iloc[win-1::win]
plt.plot(ruido3_media, label='Media')
plt.plot(ruido3_std, label='Desviación estándar')
plt.axhline(y=0, color='r', linestyle='--')
plt.title("Características estadísticas: [ruido3]")
plt.legend()
plt.ylim(-1,1)
plt.show()

Se puede percibir que esta señal **es una serie estacionaria** al tener una media con un nivel de variación prácticamente constante que hace que sus características seal independiente del eje de tiempo.

### Series temporales sintéticas

Las series temporales sintéticas (___Synthetic time series___) son datos **creados artificialmente** que se utilizan para **simular** datos del mundo real (_real-world data_), normalmente para realizar pruebas o experimentos. Estos datos de series de tiempo se pueden generar utilizando varias técnicas estadísticas y modelos matemáticos.

- #### Trend-stationary

Un proceso estacionario de tendencia (___trend-stationary___) es un proceso estocástico del que se puede eliminar una tendencia subyacente, dejando un proceso estacionario.  

Estas series temporales muestran una **tendencia estable** a largo plazo con **fluctuaciones a corto plazo** en torno a esa tendencia. Estas variaciones se mantienen de forma similar a lo largo del tiempo, pero son impredecibles y aleatorias (_ruido_).

In [ ]:
# Definir el rango o el intervalo de tiempo
inicio = "2023-03-10 00:00:00"
fin = "2023-03-20 00:00:00"
rango_tiempo = pd.date_range(inicio, fin, freq="H")
rango_tiempo

Definimos una variable como la secuencia de tiempo: _t_

In [ ]:
t = np.arange(len(rango_tiempo))
t

In [ ]:
# Fijamos la semilla
np.random.seed(77)

# Generamos un componente de tendencia (Trend)
tendencia = 1.25 * t

# Generamos un ruido blanco
ruido = 5 * np.random.randn(len(t))

# Combinamos la tendencia y el ruido
valores = tendencia + ruido

# Crear un DataFrame de la serie temporal con los valores y los índices
df_serie = pd.DataFrame(data=valores, index=rango_tiempo)
df_serie

In [ ]:
# Graficar la serie temporal sintética
plt.plot(df_serie)
plt.title("Serie temporal sintética (Trend-stationary)")
plt.xlabel("Tiempo (h)")
plt.ylabel("Valores")
plt.show()

Claramente esta serie temporal es una serie **no estacionaria**,  porque con el paso de tiempo su nivel va cambiando. Se ve que hay una clara regresión entre el valor actual y los valores de momentos anteriores.

In [ ]:
# Comprobamos la ACF
from statsmodels.graphics.tsaplots import plot_acf

rcParams['figure.figsize'] = 14, 7
plot_acf(df_serie, lags=24)
plt.xticks(np.arange(24))
plt.ylim(-1.1,1.1)
plt.show()

Se puede ver que la **autocorrelación existe** en esta serie y se disminuye según calculamos los retrasos anteriores. A cambio la **autocorrelación parcial indica una regresión prácticamente lineal** entre el valor de un momento determinado y el valor del paso anterior (_lag1_)

In [ ]:
# Comprobamos la PACF
from statsmodels.graphics.tsaplots import plot_pacf

rcParams['figure.figsize'] = 14, 7
plot_pacf(df_serie, lags=24, method='yw')
plt.xticks(np.arange(24))
plt.ylim(-1.1,1.1)
plt.show()

Podemos estudiar la evolución de las medidas estadísticas y ver que salvo la tendencia que hace que **aumente su media**, el resto de los aspectos como la varianza o **la dispersión sigue igual** a lo largo de tiempo

In [ ]:
win = 60
df_serie_media = df_serie.rolling(win).mean().iloc[win-1::win]
df_serie_std = df_serie.rolling(win).std().iloc[win-1::win]
plt.plot(df_serie_media, label='Media')
plt.plot(df_serie_std, label='Desviación estándar')
plt.axhline(y=0, color='r', linestyle='--')
plt.title("Características estadísticas: (Trend-stationary)")
plt.legend()
plt.show()

- #### Seasonal-stationary

Una serie de tipo estacionaria estacional (___seasonal-stationary___) muestra una **media estable**, una **varianza** y una estructura de **autocorrelación consistente** a lo largo del tiempo, a pesar de tener un **patrón estacional repetitivo**. En otras palabras, las propiedades estadísticas de los datos permanecen constantes a lo largo del tiempo, aunque los **valores pueden cambiar estacionalmente**.

In [ ]:
# Fijar la semilla
np.random.seed(111)

# Indicar la frecuencia (de una señal armónica commo el seno)
frecuencia = 12

# Crear las secuencias del eje de tiempo
t = np.linspace(0,1, len(rango_tiempo))

# Generar el seno
seno = 5*np.sin(2*np.pi*frecuencia*t)

# Generar un ruido blanco
ruido_normal = np.random.normal(loc=0, scale=0.8, size=len(rango_tiempo))

# Añadir el ruido
valores2 = seno + ruido_normal

# Crear un DataFrame de la serie temporal con los valores y los índices
df_serie2 = pd.DataFrame(data=valores2, index=rango_tiempo)
df_serie2

# Graficar la serie temporal sintética
plt.plot(df_serie2)
plt.ylim(-10,10)
plt.title("Serie temporal sintética: (Seasonal-stationary)")
plt.xlabel("Tiempo (h)")
plt.ylabel("Valores")
plt.show()

In [ ]:
# Sacar la gráfica interactiva
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_serie2.index,
    y=df_serie2.iloc[:,0],
    mode="markers+lines",
    ))

fig.show()

In [ ]:
# Comprobamos la ACF
from statsmodels.graphics.tsaplots import plot_acf

rcParams['figure.figsize'] = 14, 7
plot_acf(df_serie2, lags=50)
plt.xticks(np.arange(50))
plt.ylim(-1.1,1.1)
plt.show()

In [ ]:
# Comprobamos la PACF
from statsmodels.graphics.tsaplots import plot_pacf

rcParams['figure.figsize'] = 14, 7
plot_pacf(df_serie2, lags=30, method='ols')
plt.xticks(np.arange(30))
plt.ylim(-1.1,1.1)
plt.show()

In [ ]:
win = 30
df_serie2_media = df_serie2.rolling(win).mean().iloc[win-1::win]
df_serie2_std = df_serie2.rolling(win).std().iloc[win-1::win]
plt.plot(df_serie2_media, label='Media')
plt.plot(df_serie2_std, label='Desviación estándar')
plt.axhline(y=0, color='r', linestyle='--')
plt.title("Características estadísticas: (Seasonal-stationary)")
plt.ylim(-5,5)
plt.legend()
plt.show()

### Non-Stationary Time Series

#### Chirp signal

Una señal ___chirp___ es un tipo de serie temporal en la que **la frecuencia varía con el tiempo**. Las señales chirp se utilizan ampliamente en varios campos, incluidos el **radar**, el sonar, los sistemas de **comunicación** y las **imágenes biomédicas**.

In [ ]:
from scipy import signal

# Establecer los parámetros de la señal
f0 = 1  # frecuencia inicial
f1 = 250  # frecuencia final
T = 1  # Duración de la señal (segundos)

# Generar el "chirp signal"
t = np.linspace(0, T, int(T * 1000), endpoint=False)
amplitud = 5*signal.chirp(t, f0=f0, f1=f1, t1=T, method='logarithmic')
df_chirp = pd.DataFrame(data=amplitud, index=t)

# Graficar la señal
plt.plot(df_chirp)
plt.xlabel('Tiempo (s)')
plt.ylabel('Amplitud')
plt.title('Chirp Signal')
plt.show()

Como es de esperar, esta señal muestra variaciones en sus propiedades estadísticas, y por lo tanto es **un ejemplo de la no estacionariedad**.

In [ ]:
win = 50
df_chirp_media = df_chirp.rolling(win).mean().iloc[win-1::win]
df_chirp_std = df_chirp.rolling(win).std().iloc[win-1::win]
plt.plot(df_chirp_media, label='Media')
plt.plot(df_chirp_std, label='Desviación estándar')
plt.axhline(y=0, color='r', linestyle='--')
plt.title("Características estadísticas: (Chirp Signal)")
plt.ylim(-8,8)
plt.legend()
plt.show()

Los datos no estacionarios pueden presentar por ejemplo **cambios en la dispersión** o en la varianza de sus valores.

In [ ]:
# Fijar la semilla
np.random.seed(333)

# Determinar el número de observaciones
n = 10000

# Introducir la desviación estándar inicial y el incremento que se le realiza en cada paso
std_inicial = 1.0
std_incremento = 0.0002

# Generar los índices
start_date = pd.Timestamp('2000-01-01')
indices = pd.date_range(start_date, periods=n, freq='D')

# Crear el DataFrame
datos = np.zeros(n)
for i in range(1, n):
    # Incrementos cuadráticos de la desviación estándar de los valores de la serie
    desv_estandar = std_inicial + std_incremento * (i+1)**2
    datos[i] = np.random.normal(loc=0, scale=desv_estandar)
df_std_creciente = pd.DataFrame(datos, index=indices)

# Graficar la serie sintética
plt.plot(df_std_creciente)
plt.xlabel('Tiempo')
plt.ylabel('Valores')
plt.title('Serie temporal con aumento cuadrático en la desviación estándar')
plt.show()


In [ ]:
# Comprobamos la ACF
from statsmodels.graphics.tsaplots import plot_acf

rcParams['figure.figsize'] = 14, 7
plot_acf(df_std_creciente, lags=50)
plt.xticks(np.arange(50))
plt.ylim(-1.1,1.1)
plt.show()

In [ ]:
win = 100
df_std_creciente_media = df_std_creciente.rolling(win).mean().iloc[win-1::win]
df_std_creciente_std = df_std_creciente.rolling(win).std().iloc[win-1::win]
plt.plot(df_std_creciente_media, label='Media')
plt.plot(df_std_creciente_std, label='Desviación estándar')
plt.axhline(y=0, color='r', linestyle='--')
plt.title("Características estadísticas: (Non-stationary)")
plt.legend()
plt.show()

### Differencing

La **diferenciación** (_Differencing_) es un método para **transformar** un conjunto de datos de series temporales. Se puede usar para **eliminar la dependencia temporal** de la serie. Esto incluye estructuras como **tendencias** y **estacionalidad**.  

La diferenciación de **primer grado** de una serie temporal se define como la **diferencia entre observaciones consecutivas**.

Transformamos la serie temporal sintética que era estacionaria de tendencia (_trend-stationary_) aplicando una diferenciación del primer orden:

In [ ]:
# Graficar la serie temporal sintética
plt.plot(df_serie)
plt.title("Serie temporal sintética (Trend-stationary)")
plt.xlabel("Tiempo (h)")
plt.ylabel("Valores")
plt.show()

In [ ]:
# Graficar la serie temporal sintética diferenciada
plt.plot(df_serie.diff())
plt.title("Serie temporal sintética diferenciada (Trend-stationary) - [diff(1)]")
plt.xlabel("Tiempo (h)")
plt.ylabel("Valores")
plt.show()

In [ ]:
# Comprobamos la ACF
from statsmodels.graphics.tsaplots import plot_acf

rcParams['figure.figsize'] = 14, 7
plot_acf(df_serie.diff().dropna(), lags=24)
plt.xticks(np.arange(24))
plt.ylim(-1.1,1.1)
plt.show()

In [ ]:
win = 20
df_serie_media_diff = df_serie.diff().rolling(win).mean().iloc[win-1::win]
df_serie_std_diff = df_serie.diff().rolling(win).std().iloc[win-1::win]
plt.plot(df_serie_media_diff, label='Media')
plt.plot(df_serie_std_diff, label='Desviación estándar')
plt.axhline(y=1.25, color='r', linestyle='--')
plt.ylim(-10,10)
plt.title("Características estadísticas: (Trend-stationary) - [diff(1)]")
plt.legend()
plt.show()

Se puede observar que tras la diferenciación se estabilizan las medidas estadísticas.

In [ ]:
# Graficamos la serie temporal sintética
plt.plot(df_serie2)
plt.ylim(-10,10)
plt.title("Serie temporal sintética: (Seasonal-stationary)")
plt.xlabel("Tiempo (h)")
plt.ylabel("Valores")
plt.show()

In [ ]:
# Graficamos la serie temporal sintética
plt.plot(df_serie2.diff())
plt.ylim(-10,10)
plt.title("Serie temporal sintética: (Seasonal-stationary) - [diff(1)]")
plt.xlabel("Tiempo (h)")
plt.ylabel("Valores")
plt.show()

In [ ]:
# Comprobamos la ACF
from statsmodels.graphics.tsaplots import plot_acf

rcParams['figure.figsize'] = 14, 7
plot_acf(df_serie2.diff().dropna(), lags=50)
plt.xticks(np.arange(50))
plt.ylim(-1.1,1.1)
plt.show()

Podemos comprobar que la serie temporal diferenciada **todavía muestra cambios en sus propiedades** en el eje de tiempo, como un **patrón estacional** y **autocorrelación**. Por lo tanto procedemos con la diferenciación del segundo orden.

In [ ]:
# Graficamos la serie temporal sintética
plt.plot(df_serie2.diff().diff())
plt.ylim(-10,10)
plt.title("Serie temporal sintética: (Seasonal-stationary) - [diff(2)]")
plt.xlabel("Tiempo (h)")
plt.ylabel("Valores")
plt.show()

In [ ]:
# Comprobamos la ACF
from statsmodels.graphics.tsaplots import plot_acf

rcParams['figure.figsize'] = 14, 7
plot_acf(df_serie2.diff().diff().dropna(), lags=50)
plt.xticks(np.arange(50))
plt.ylim(-1.1,1.1)
plt.show()

In [ ]:
win = 30
df_serie2_media_diff2 = df_serie2.diff().diff().rolling(win).mean().iloc[win-1::win]
df_serie2_std_diff2 = df_serie2.diff().diff().rolling(win).std().iloc[win-1::win]
plt.plot(df_serie2_media_diff2, label='Media')
plt.plot(df_serie2_std_diff2, label='Desviación estándar')
plt.axhline(y=0, color='r', linestyle='--')
plt.title("Características estadísticas: (Seasonal-stationary) - [diff(2)]")
plt.ylim(-5,5)
plt.legend()
plt.show()

En este caso se puede apreciar que al realizar dos veces la diferenciación se consigue una señal estacionaria.

Ahora volvemos al caso de "_chirp signal_" que también era un ejemplo de una serie no estacionaria:

In [ ]:
win = 50
df_chirp_media = df_chirp.rolling(win).mean().iloc[win-1::win]
df_chirp_std = df_chirp.rolling(win).std().iloc[win-1::win]
plt.plot(df_chirp_media, label='Media')
plt.plot(df_chirp_std, label='Desviación estándar')
plt.axhline(y=0, color='r', linestyle='--')
plt.title("Características estadísticas: (Chirp Signal)")
plt.ylim(-8,8)
plt.legend()
plt.show()

Realizamos la diferenciación del primer orden:

In [ ]:
# Graficar la señal diferenciada
plt.plot(df_chirp.diff())
plt.xlabel('Tiempo (s)')
plt.ylabel('Amplitud')
plt.title('Chirp Signal - [diff(1)]')
plt.show()

In [ ]:
win = 50
df_chirp_media_diff = df_chirp.diff().rolling(win).mean().iloc[win-1::win]
df_chirp_std_diff = df_chirp.diff().rolling(win).std().iloc[win-1::win]
plt.plot(df_chirp_media_diff, label='Media')
plt.plot(df_chirp_std_diff, label='Desviación estándar')
plt.axhline(y=0, color='r', linestyle='--')
plt.title("Características estadísticas: (Seasonal-stationary) - [diff(1)]")
plt.legend()
plt.show()

Se puede confirmar que en este caso la diferenciación **elimina las oscilaciones del nivel de la serie**, sin embargo, **no consigue estabilizar la varianza** de los datos.  

Vamos a analizar el caso de la serie temporal con la **varianza creciente**:

In [ ]:
# Graficar la serie sintética diferenciada
plt.plot(df_std_creciente.diff())
plt.xlabel('Tiempo')
plt.ylabel('Valores')
plt.title('Serie temporal con aumento cuadrático en la desviación estándar - [diff(1)]')
plt.show()


In [ ]:
win = 100
df_std_creciente_media_diff = df_std_creciente.diff().rolling(win).mean().iloc[win-1::win]
df_std_creciente_std_diff = df_std_creciente.diff().rolling(win).std().iloc[win-1::win]
plt.plot(df_std_creciente_media_diff, label='Media')
plt.plot(df_std_creciente_std_diff, label='Desviación estándar')
plt.axhline(y=0, color='r', linestyle='--')
plt.title("Características estadísticas: (Non-stationary)  - [diff(1)]")
plt.legend()
plt.show()

Se puede ver que la diferenciación consigue de nuevo **disminuir las alteraciones en la media** de la serie, pero **no logra estabilizar la varianza**, aunque sigamos con **más ordenes de diferenciación**.

In [ ]:
win = 100
df_std_creciente_media_diff2 = df_std_creciente.diff().diff().rolling(win).mean().iloc[win-1::win]
df_std_creciente_std_diff2 = df_std_creciente.diff().diff().rolling(win).std().iloc[win-1::win]
plt.plot(df_std_creciente_media_diff2, label='Media')
plt.plot(df_std_creciente_std_diff2, label='Desviación estándar')
plt.axhline(y=0, color='r', linestyle='--')
plt.title("Características estadísticas: (Non-stationary)  - [diff(2)]")
plt.legend()
plt.show()

### Variable transformations

La transformación de variables consiste en reemplazar los valores originales de las variables con una función de esa variable. Las transformaciones con funciones matemáticas ayudan a **reducir el sesgo** y también **mejorar la distribución** de los valores.

Hay varias transformaciones que se pueden aplicar a los datos de series temporales para estabilizar la varianza. Estas transformaciones se conocen comúnmente como transformaciones estabilizadoras de varianza (___variance stabilizing transformations___) que intentan llevar la distribución de la variable a una forma más simétrica, o en otras palabras, Gaussiana que incluyen:

- **Transformación logarítmica**: Consiste en **tomar el logaritmo natural de los valores** de la serie temporal. Es útil cuando la varianza de la serie temporal aumenta con el nivel de la serie.

- **Transformación de raíz cuadrada** (_square root_): esto implica **sacar la raíz cuadrada de los valores** en la serie de tiempo. Es útil cuando la varianza de la serie temporal aumenta con el nivel de la serie.

- **Transformación de Box-Cox**: Esta es **una familia de transformaciones de potencia** que incluye las transformaciones logarítmicas y de raíz cuadrada como casos especiales. La transformación de Box-Cox se puede utilizar para encontrar el mejor parámetro de transformación lambda que estabilice la varianza de la serie temporal.

- **Transformación Yeo-Johnson**: La transformación de Yeo-Johnson fue introducida en **2000** por Yeo and Johnson como **una extensión de la transformación de Box-Cox**. Mientras que la transformación de **Box-Cox requiere que los datos sean positivos**, la transformación de **Yeo-Johnson puede manejar tanto datos positivos como negativos**.

Vamos a aplicar este último método para transformar la serie temporal de pasageros aéreos que vimos al principio:

In [ ]:
# Visualizamos la serie de ejemplo
sns.lineplot(data=df_air, x=df_air.index, y=df_air['value'])
plt.title("Número de pasajeros aéreos de cada mes desde el año 1949 al 1960")
plt.show()

Dibujamos la evolución de los atributos estadísticos:

In [ ]:
win = 20
df_air_media = df_air.rolling(win).mean().iloc[win-1::win]
df_air_std = df_air.rolling(win).std().iloc[win-1::win]
plt.plot(df_air_media, label='Media')
plt.plot(df_air_std, label='Desviación estándar')
plt.axhline(y=0, color='r', linestyle='--')
plt.title("Características estadísticas: (Air Passengers)")
plt.legend()
plt.show()

Se puede apreciar una tendencia creciente tanto en la medoia como en la varianza de los valores de la serie temporal. Procedemos a realizar la diferenciación del primer grado:

In [ ]:
# Graficar la señal diferenciada
plt.plot(df_air.diff())
plt.xlabel('Tiempo (mes)')
plt.ylabel('Pasageros')
plt.title('(Air Passengers) - [diff(1)]')
plt.show()

In [ ]:
win = 20
df_air_media_diff = df_air.diff().rolling(win).mean().iloc[win-1::win]
df_air_std_diff = df_air.diff().rolling(win).std().iloc[win-1::win]
plt.plot(df_air_media_diff, label='Media')
plt.plot(df_air_std_diff, label='Desviación estándar')
plt.axhline(y=0, color='r', linestyle='--')
plt.title("Características estadísticas: (Air Passengers)  - [diff(1)]")
plt.legend()
plt.show()

Conseguimos estabilizar la media pero la varianza sigue creciendo en el eje de tiempo. Para lograr el mismo efecto sobre la variabilidad de los datos, aplicamos una transformada de tipo **`Box-Cox`** por disponer de valores positivos.

In [ ]:
from scipy.stats import boxcox

# Aplicar la transformación
data_air_bc, lambda_air = boxcox(df_air.values.reshape(-1))

# Graficar los datos transformados
plt.plot(data_air_bc)
plt.title("Datos transformados por Cox-Box")
plt.xlabel("Tiempo (mes)")
plt.ylabel("Pasageros")
plt.show()

print(lambda_air)


In [ ]:
win = 10
df_air_bc_media = pd.DataFrame(data_air_bc).rolling(win).mean().iloc[win-1::win]
df_air_bc_std = pd.DataFrame(data_air_bc).rolling(win).std().iloc[win-1::win]
plt.plot(df_air_bc_media, label='Media')
plt.plot(df_air_bc_std, label='Desviación estándar')
plt.axhline(y=0, color='r', linestyle='--')
plt.title("Características estadísticas: (Air Passengers)  - [Box-Cox]")
plt.legend()
plt.show()

Podemos usar la función `inv_boxcox` para realizar un **cálculo inverso** y volver a tener los **valores originales**.

In [ ]:
from scipy.special import inv_boxcox
data_bc_inv = inv_boxcox(data_air_bc, lambda_air)

plt.plot(data_bc_inv)
plt.plot()


---

### **`Ejercicio 24.1`**

Vamos a nalizar los datos de **`Sunspots Dataset`** que son números promediados mensuales de **manchas solares desde 1749 hasta 1983**. Esta serie tempòral se ha recolectado en el _Observatorio Federal Suizo_ (_**Zúrich**_) hasta **1960**, luego en el _Observatorio Astronómico de Japón_ (**_Tokio_**).  

Los números de manchas solares son una *medida de la actividad solar*. Los datos vienen en las unidades originales utilizadas por los observadores de forma bruta y **no hay ajustes** por cambios en la instrumentación o sesgo del observador.


**`24.1.1`** Utiliza el siguiente enlace para descargar los datos y crear una tabla tipo _DataFrame_ con ellos:
 - 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-sunspots.csv'

**`24.1.2`** construye la serie temporal, del modo que las fechas tipo `'1749-05-01'` formen los índices, y número de las manchas solares los valores de la serie.

**`24.1.3`** Representa de forma gráfica los datos de la serie temporal con las siguientes visualizaciones:
 - Gráfica estática (_matplotlib_)
 - Gráfica interactiva (_plotly_)

**`24.1.4`** Aplica una descomposición de la serie temporal mediante el método `seasonal_decompose` y visualiza los componentes correspondientes.

**`24.1.5`** Aplica una descomposición de la serie temporal mediante el método `STL` y visualiza los componentes correspondientes.

**`24.1.6`** Saca la gráfica de ***correlograma*** para la serie temporal, considerando hasta el retraso indicado y analiza los patrones que ves presente en esta gráfica:

 - `lags=200`

**`24.1.7`** Saca la gráfica de ***Autocorrelación parcial*** para la serie temporal, considerando hasta el retraso indicado y analiza los patrones que ves presente en esta gráfica:

 - `lags=30`

**`24.1.8`** Saca la gráfica de las ***Características estadísticas*** y su evolución a lo largo de tiempo de **la serie temporal**, considerando una ventana con el tamaño indicado. Analiza y explica esta evolución de la media y de la varianza de los datos:

 - `win = 200`

**`24.1.9`** Transforma los datos de la serie temporal mediante la transformada de `Yeo-Johnson` que se aplica de una forma muy similar a la transformación `Box-Cox`. Después, visualiza los datos transformados e imprime el valor de la potencia o el hiperparámetro que se haya aplicado en la transformación (_lambda_).

**`24.1.10`** Saca la gráfica de las ***Características estadísticas*** y su evolución a lo largo de tiempo de **la serie temporal transformada**, considerando una ventana con el tamaño indicado. Analiza y explica esta evolución de la media y de la varianza de los datos:

 - `win = 200`

**`24.1.11`** Aplica una **diferenciación del primer orden** sobre estos datos transformados anteriormente. Saca la gráfica de la ***la serie temporal transformada y diferenciada***.

**`24.1.12`** Saca la gráfica de las ***Características estadísticas*** y su evolución a lo largo de tiempo de **la serie temporal transformada y diferenciada**, considerando una ventana con el tamaño indicado. Analiza y explica esta evolución de la media y de la varianza de los datos:

 - `win = 200`

---